# Step 1: Import Data and General Check 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10, 7)

In [2]:
df = pd.read_csv(r'C:\Users\runzh\OneDrive\Desktop\IMDBdata_MainData.csv')

In [ ]:
display(df.head(5))

In [3]:
df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response', 'tomatoURL'],
      dtype='object')

In [4]:
df.shape

(5273, 27)

In [5]:
# check data types
# Released column has to be changed
# BoxOffice column has to be changed 
# Runtime column has to be changed

print(df.dtypes)

Title              object
Year                int64
Rated              object
Released           object
Runtime            object
Genre              object
Director           object
Writer             object
Actors             object
Plot               object
Language           object
Country            object
Awards             object
Poster             object
Ratings.Source     object
Ratings.Value      object
Metascore         float64
imdbRating        float64
imdbVotes          object
imdbID             object
Type               object
DVD                object
BoxOffice          object
Production         object
Website            object
Response             bool
tomatoURL          object
dtype: object


In [6]:
# check the unique value of movie title
# some repetitive movies but only a small fraction...

print(df['Title'].nunique())

5261


In [7]:
# check missing values 

print(df.isnull().sum())

Title                0
Year                 0
Rated              458
Released            38
Runtime             16
Genre                2
Director             1
Writer              52
Actors               7
Plot                11
Language            10
Country              2
Awards             811
Poster              13
Ratings.Source       0
Ratings.Value        0
Metascore         1397
imdbRating           0
imdbVotes            0
imdbID               0
Type                 0
DVD                391
BoxOffice         2639
Production         328
Website           1801
Response             0
tomatoURL           86
dtype: int64


# Step 2: Transform Data

In [9]:
# the Runtime of row 4207 when Title is Låt den rätte komma ut is 1 hour 27 minute
# need to change Runtime to 87 minutes to keep the unit aligned

df.loc[df['Title']=='Låt den rätte komma ut', 'Runtime'] = '87 min'
row = df.iloc[4207]
print(row)

Title                                        Låt den rätte komma ut
Year                                                           2010
Rated                                                           NaN
Released                                                        NaN
Runtime                                                      87 min
Genre                                        Action, Comedy, Horror
Director                                                 Victor Säf
Writer                                                   Victor Säf
Actors            Lukas Ingström, Kenny Hegg, Emil Gunnarsson, V...
Plot              Adam wakes up with his arms chained to the cei...
Language                                                        NaN
Country                                                         NaN
Awards                                                          NaN
Poster                                                          NaN
Ratings.Source                              Inte

In [8]:
# only extract the number part in the string before space 
# convert string to number 
# replace Runtime with 0 when it is NaN 

df['Runtime_2']=df['Runtime'].str.split().str[0]
df['Runtime_2'] = pd.to_numeric(df['Runtime_2'], errors='coerce')
df['Runtime_2'] = df['Runtime_2'].fillna(0).astype(int)

In [9]:
print(df['Runtime'].dtype)
print(df['Runtime_2'].dtype)

object
int32


In [12]:
# check the type of Runtime_2

row = df.iloc[4]
print(row)

Title                                                       Manos: The Hands of Fate
Year                                                                            1966
Rated                                                                      NOT RATED
Released                                                                   15-Nov-66
Runtime                                                                       70 min
Genre                                                                         Horror
Director                                                            Harold P. Warren
Writer                                                 Harold P. Warren (screenplay)
Actors                             Tom Neyman, John Reynolds, Diane Adelson, Haro...
Plot                               A family gets lost on the road and stumbles up...
Language                                                                     English
Country                                                          

In [11]:
# get the number of actors for each movie
# get the number of directors for each movie
# get the total number of people involved for each movie

df['Number of Actors']=df['Actors'].str.count(',')+1
df['Number of Director']=df['Director'].str.count(',')+1
df['Total Number of People Involved']=df['Number of Actors']+df['Number of Director']

In [13]:
import datetime

df['Released_2'] = pd.to_datetime(df['Released'],format='%d-%b-%y', errors='coerce')
current_year = datetime.datetime.now().year
df['Released_2'] = df['Released_2'].apply(lambda x: x.replace(year=x.year - 100) if x.year > current_year else x)
df['Released_2'] = df['Released_2'].dt.date

In [14]:
# split genre to multiple columns
# one-hot encode the genre column

df['Genre'].fillna('N/A', inplace = True)
df['Genre_List'] = df['Genre'].str.split(',').apply(lambda x: [item.strip() for item in x])
exploded_df = df.explode('Genre_List')
one_hot = pd.get_dummies(exploded_df['Genre_List'])
one_hot_encoded_df = one_hot.groupby(exploded_df.index).max()

In [16]:
# genre column can be one-hot encoded to 25 distinct columns including N/A...

display(one_hot_encoded_df.columns)
display(one_hot_encoded_df.shape)

Index(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'N/A', 'News', 'Romance',
       'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')

(5273, 25)

In [15]:
# second way to split genre and get counts of each directly

genres_series = df['Genre'].str.split(',\s*').explode()

# Count the occurrences of each genre
genre_counts = genres_series.value_counts().reset_index()

# Rename columns
genre_counts.columns = ['genre', 'count']
genre_counts.head()

,genre,count
0,Drama,2738
1,Comedy,1939
2,Action,1217
3,Adventure,953
4,Crime,930


In [44]:
genre_counts.to_csv(r'C:\Users\runzh\OneDrive\Desktop\genre_counts', index=False)

In [16]:
# split language to multiple columns
# one-hot encode the language column

df['Language'].fillna('N/A', inplace = True)
df['Language_List'] = df['Language'].str.split(',').apply(lambda x: [item.strip() for item in x])
exploded_df_2 = df.explode('Language_List')
one_hot_2 = pd.get_dummies(exploded_df_2['Language_List'])
one_hot_encoded_df_2 = one_hot_2.groupby(exploded_df_2.index).max()

In [18]:
display(one_hot_encoded_df_2.columns)
display(one_hot_encoded_df_2.shape)

Index(['Aboriginal', 'Acholi', 'Afrikaans', 'Albanian', 'Algonquin',
       'American Sign Language', 'Amharic', 'Ancient (to 1453)',
       'Apache languages', 'Arabic',
       ...
       'Turkish', 'Ukrainian', 'Urdu', 'Vietnamese', 'Washoe', 'Welsh',
       'Wolof', 'Xhosa', 'Yiddish', 'Zulu'],
      dtype='object', length=150)

(5273, 150)

In [17]:
# second way to split language and get counts of each directly

Language_series = df['Language'].str.split(',\s*').explode()

# Count the occurrences of each genre
Language_counts = Language_series.value_counts().reset_index()

# Rename columns
Language_counts.columns = ['Language', 'count']
Language_counts.head()

,Language,count
0,English,4839
1,Spanish,484
2,French,476
3,German,283
4,Hindi,265


In [46]:
Language_counts.to_csv(r'C:\Users\runzh\OneDrive\Desktop\Language_counts', index=False)

In [20]:
# split country to multiple columns
# one-hot encode the country column

df['Country'].fillna('N/A', inplace = True)
df['Country_List'] = df['Country'].str.split(',').apply(lambda x: [item.strip() for item in x])
exploded_df_3 = df.explode('Country_List')
one_hot_3 = pd.get_dummies(exploded_df_3['Country_List'])
one_hot_encoded_df_3 = one_hot_3.groupby(exploded_df_3.index).max()

In [21]:
display(one_hot_encoded_df_3.columns)
display(one_hot_encoded_df_3.shape)

Index(['Afghanistan', 'Algeria', 'Angola', 'Antarctica', 'Argentina', 'Aruba',
       'Australia', 'Austria', 'Bahamas', 'Belgium', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cambodia', 'Cameroon',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt',
       'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Guadeloupe', 'Guyana', 'Hong Kong', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Isle Of Man', 'Israel',
       'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait',
       'Kyrgyzstan', 'Latvia', 'Lebanon', 'Libya', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'Malaysia', 'Malta', 'Mexico', 'Monaco',
       'Morocco', 'N/A', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway',
       'Pakistan', 'Panama', 'Peru', 'Philippines', 'Poland', 'Puerto Rico',
       'Romania', 'Russia', 'Sa

(5273, 99)

In [22]:
# second way to split country and get counts of each directly

Country_series = df['Country'].str.split(',\s*').explode()

# Count the occurrences of each genre
Country_counts = Country_series.value_counts().reset_index()

# Rename columns
Country_counts.columns = ['Country', 'count']
Country_counts.head()

,Country,count
0,USA,4298
1,UK,762
2,Germany,411
3,France,393
4,Canada,350


In [50]:
Country_counts.to_csv(r'C:\Users\runzh\OneDrive\Desktop\Country_counts', index=False)

In [23]:
# split director to multiple columns
# one-hot encode the director column

df['Director'].fillna('N/A', inplace = True)
df['Director_List'] = df['Director'].str.split(',').apply(lambda x: [item.strip() for item in x])
exploded_df_4 = df.explode('Director_List')
one_hot_4 = pd.get_dummies(exploded_df_4['Director_List'])
one_hot_encoded_df_4 = one_hot_4.groupby(exploded_df_4.index).max()

In [24]:
display(one_hot_encoded_df_4.columns)
display(one_hot_encoded_df_4.shape)

Index(['A. Raven Cruz', 'A.R. Murugadoss', 'Aamir Khan', 'Aanand L. Rai',
       'Aaron Hann', 'Aaron Schneider', 'Aaron Seltzer', 'Aaron T. Wells',
       'Abbas Alibhai Burmawalla', 'Abbas Tyrewala',
       ...
       'Zak Penn', 'Zal Batmanglij', 'Zoran Lisinac', 'Zoya Akhtar',
       'Àlex Pastor', 'Álex de la Iglesia', 'Çagan Irmak', 'Émile Gaudreault',
       'Éric Tessier', 'Étienne Faure'],
      dtype='object', length=2932)

(5273, 2932)

In [25]:
# second way to split director and get counts of each directly

Director_series = df['Director'].str.split(',\s*').explode()

# Count the occurrences of each genre
Director_counts = Director_series.value_counts().reset_index()

# Rename columns
Director_counts.columns = ['Director', 'count']
Director_counts.head()

,Director,count
0,Steven Spielberg,27
1,Woody Allen,22
2,Martin Scorsese,21
3,Clint Eastwood,20
4,Robert Rodriguez,17


In [48]:
Director_counts.to_csv(r'C:\Users\runzh\OneDrive\Desktop\Director_counts', index=False)

In [26]:
# merge several one-hot encode dataframes with the original datasets

result_df_5 = pd.concat([df, one_hot_encoded_df, one_hot_encoded_df_2, one_hot_encoded_df_3, one_hot_encoded_df_4], axis=1)

In [28]:
display(result_df_5.shape)
display(df.shape)

(5273, 3242)

(5273, 36)

In [31]:
# convert box office from string to numeric

result_df_5['Box Office_2'] = result_df_5['BoxOffice'].str.replace('[\$,]', '', regex=True)
result_df_5['Box Office_2'] = pd.to_numeric(result_df_5['Box Office_2'], errors='coerce')

display(result_df_5['BoxOffice'].dtype)
display(result_df_5['Box Office_2'].dtype)
display(result_df_5['Box Office_2'].sum())

dtype('O')

dtype('float64')

130255146582.0

In [32]:
# convert imbd rating to numeric

result_df_5['IMBD Vote_2'] = result_df_5['imdbVotes'].str.replace(',', '', regex=True)
result_df_5['IMBD Vote_2'] = pd.to_numeric(result_df_5['IMBD Vote_2'])

display(result_df_5['imdbVotes'].dtype)
display(result_df_5['IMBD Vote_2'].dtype)
display(result_df_5['IMBD Vote_2'].max())
display(result_df_5['IMBD Vote_2'].min())
display(result_df_5['IMBD Vote_2'].mean())

dtype('O')

dtype('int64')

1825626

6

86458.16669827423

In [35]:
column_names = list(result_df_5.columns)
print(column_names)

['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response', 'tomatoURL', 'Runtime_2', 'Number of Actors', 'Number of Director', 'Total Number of People Involved', 'Released_2', 'Genre_List', 'Language_List', 'Country_List', 'Director_List', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'N/A', 'News', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western', 'Aboriginal', 'Acholi', 'Afrikaans', 'Albanian', 'Algonquin', 'American Sign Language', 'Amharic', 'Ancient (to 1453)', 'Apache languages', 'Arabic', 'Aramaic', 'Armenian', 'Assamese', 'Assyrian Neo-Aramaic', 'Awadhi', 'Bambara', 'Belarusian', 'Bengali', 'Berber la

In [37]:
# Top rated movies

subset = result_df_5[['Title','imdbRating']]

In [63]:
df_sorted.to_csv(r'C:\Users\runzh\OneDrive\Desktop\top_rated_movies', index=False)

In [257]:
# Final output 

result_df_5.to_csv(r'C:\Users\runzh\OneDrive\Desktop\IMBD_final_dataset.csv', index=False)

In [ ]:
# End